<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [32]:
if IN_COLAB:
    !pip install openai
    !pip install requests

from openai import OpenAI

if IN_COLAB:
    from google.colab import userdata
    OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")
else:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

### load files into Colab

NOTE: you must refresh your Colab files directory to see changes

In [9]:
import json

if IN_COLAB:
    !git clone "https://github.com/samgregson/GHPT-experiments"
    with open('/content/GHPT-experiments/components.json', 'r') as f:
        components_dict = json.load(f)
    print(components_dict)
else:
    with open('../data/components.json', 'r') as f:
        components_dict = json.load(f)
    print(components_dict)

name_iterator = (component["Name"] for component in components_dict)

[{'Name': 'Cull Pattern', 'Description': 'Cull (remove) elements in a list using a repeating bit mask.', 'IsCoreComponent': True, 'Inputs': [{'Name': 'List', 'DataType': 'Generic Data', 'IsOptional': False}, {'Name': 'Cull Pattern', 'DataType': 'Boolean', 'IsOptional': False}], 'Outputs': [{'Name': 'List', 'DataType': 'Generic Data', 'IsOptional': False}]}, {'Name': 'Brep Edges', 'Description': 'Extract the edge curves of a brep.', 'IsCoreComponent': True, 'Inputs': [{'Name': 'Brep', 'DataType': 'Brep', 'IsOptional': False}], 'Outputs': [{'Name': 'Naked', 'DataType': 'Curve', 'IsOptional': False}, {'Name': 'Interior', 'DataType': 'Curve', 'IsOptional': False}, {'Name': 'Non-Manifold', 'DataType': 'Curve', 'IsOptional': False}]}, {'Name': 'Char Sequence', 'Description': 'Create a sequence of textual characters.', 'IsCoreComponent': True, 'Inputs': [{'Name': 'Count', 'DataType': 'Integer', 'IsOptional': False}, {'Name': 'Char Pool', 'DataType': 'Text', 'IsOptional': False}, {'Name': 'For

# OpenAI

In [71]:

client = OpenAI(api_key=OPENAI_API_KEY)

if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# GHPT

### Import prompts

In [4]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

In [5]:
# example use:
prompt_template.format(QUESTION="create a sphere")

'\n// Question : How do I add two numbers inside of Grasshopper?\n// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together\n// JSON:\n{\n\t"Advice": "Make sure to set the number sliders to the correct value",\n\t"Additions": [\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 1,\n\t\t\t"value": "0..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 2,\n\t\t\t"value": "-50..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Addition",\n\t\t\t"Id": 3\n\t\t}\n\t],\n\t"Connections": [\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "A"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 1,\n\t\t\t\t"ParameterName": "number"\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "B"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 2,\n\t\t\t\t"Parame

In [6]:
system_prompt

"\nYou are a Grasshopper Expert and are going to help create Grasshopper Definitions.\nKeep the answers short and concise.\nMake sure you create and connect a component for every non-optional input\nIf you're not sure about the answer, but think there's additional information that could help you, please ask for that information.\n\nAlways use the given format, avoid any devitation.\n"

# Instructor

In [42]:
if IN_COLAB:
    !pip install -U instructor
    !pip install anthropic

In [61]:
from pydantic import BaseModel, Field, field_validator, ValidationError
from typing import List, Optional

class Component(BaseModel):
    Name: str = Field(..., description="The name of the component to be added. Only standard grasshopper components are allowed")
    Id: int = Field(..., description="A unique identifier for the component, starting from 1 and counting upwards")
    Value: Optional[str] = Field(None, alias='Value', description="The range of values for the component, if applicable. Only to be used for Panel, Number Slider, or Point components")

class InputConnectionDetail(BaseModel):
    Id: int = Field(..., description="The unique identifier of the component the connection is related to")
    ParameterName: str = Field(..., description="The specific input parameter of the component that the connection affects")

class OutputConnectionDetail(BaseModel):
    Id: int = Field(..., description="The unique identifier of the component the connection is related to")
    ParameterName: str = Field(..., description="The specific output parameter of the component that the connection affects")

class Connection(BaseModel):
    From: OutputConnectionDetail = Field(..., description="The source component and parameter from which the connection originates")
    To: InputConnectionDetail = Field(..., description="The target component and parameter that the connection is directing to")

class GrasshopperScriptModel(BaseModel):
    """
    A representation of a grasshopper script with all grasshopper components and the connections between them. Use sliders for variable inputs to the script
    """
    ChainOfThought: str = Field(..., description="step by step rational explaining how the script acheives the aim, including the main components used")
    Advice: str = Field(..., description="A piece of advice or instruction related to using the grasshopper script")
    Additions: List[Component] = Field(..., description="A list of components to be added to the configuration")
    Connections: List[Connection] = Field(..., description="A list of connections defining relationships between components' parameters")

    @field_validator("ChainOfThought")
    @classmethod
    def validate_cot(cls, v):
        if len(v.split()) > 20:
            raise ValueError("ChainOfThought response must max 30 words. Keep it short and concise.")
        return v

# Example usage:
example_json = {
    "ChainOfThought": "Use the sphere component with a slider as input to set the radius",
    "Advice": "Adjust the radius of the sphere using a number slider for desired size",
    "Additions": [
        {
            "Name": "Sphere",
            "Id": 1
        },
        {
            "Name": "Number Slider",
            "Id": 2,
            "Value": "5..50..100"
        }
    ],
    "Connections": [
        {
            "To": {
                "Id": 1,
                "ParameterName": "Radius"
            },
            "From": {
                "Id": 2,
                "ParameterName": "Number"
            }
        }
    ]
}

try:
    config = GrasshopperScriptModel(**example_json)
    print(config.model_dump())
except ValidationError as e:
    print(e)

{'ChainOfThought': 'Use the sphere component with a slider as input to set the radius', 'Advice': 'Adjust the radius of the sphere using a number slider for desired size', 'Additions': [{'Name': 'Sphere', 'Id': 1, 'Value': None}, {'Name': 'Number Slider', 'Id': 2, 'Value': '5..50..100'}], 'Connections': [{'From': {'Id': 2, 'ParameterName': 'Number'}, 'To': {'Id': 1, 'ParameterName': 'Radius'}}]}


In [47]:
import instructor

client_instructor = instructor.patch(client)

In [70]:
# create the openai api call function
def call_openai_instructor(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client_instructor.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_model=GrasshopperScriptModel,
        max_retries = 1
    )

    return completion

In [62]:
model = "gpt-3.5-turbo-1106"
# model = "gpt-4-1106-preview"
response = call_openai_instructor(prompt='sphere', system_prompt=system_prompt)

In [63]:
print(response.json())

{"ChainOfThought":"Create a sphere using the Sphere component and set its radius using a Number Slider.","Advice":"Make sure to set the radius of the sphere to the desired value using the Number Slider.","Additions":[{"Name":"Sphere","Id":1,"Value":null},{"Name":"Number Slider","Id":2,"Value":"0 to 10"}],"Connections":[{"From":{"Id":2,"ParameterName":"Value"},"To":{"Id":1,"ParameterName":"Radius"}}]}
